In [1]:
import pandas as pd
df=pd.read_csv('/content/sports_dataset_uncleaned.csv')

Identify and remove duplicate rows in the dataset.

In [2]:
t=df[df.duplicated()]
df = df.drop_duplicates()
t



,Player_ID,Player_Name,Gender,Age,Country,Team,Position,Sport_Type,Match_ID,Tournament,...,Bonus,Sponsorship_Amount,Ticket_Sales,Fan_ID,Ticket_Type,Payment_Mode,Feedback_Score,Coach_Name,Win_Status,Weather


Identify and remove duplicate Player_ID values.



In [3]:
df[df['Player_ID'].duplicated()]
df = df.drop_duplicates(subset='Player_ID')

Identify and remove duplicate Match_ID entries.

In [4]:
df[df['Match_ID'].duplicated()]
df = df.drop_duplicates(subset='Match_ID')

Identify duplicate player names with different countries (possible mismatched records).



In [5]:
t = df.groupby('Player_Name')['Country'].nunique()
u = t[t > 1]
u


,Country
Player_Name,
Amit,24
Carlos,26
David,28
John,29
Liu,31
Maria,22
Mike,29
Priya,24
Sara,27


Fill missing Coach_Name values with "Unknown Coach".



In [6]:
df['Coach_Name'] = df['Coach_Name'].fillna('Unknown Coach')


Fill missing Country values using the team’s most common country.

In [7]:
t = df.groupby('Team')['Country'].agg(pd.Series.mode)
df['Country'] = df['Country'].fillna(df['Team'].map(t))



Fill missing Feedback_Score values with the median feedback per sport type.



In [8]:
t = df.groupby('Sport_Type')['Feedback_Score'].median()
df['Feedback_Score'] = df['Feedback_Score'].fillna(df['Sport_Type'].map(t))

Fill missing Bonus values with the average bonus of the player’s team.



In [9]:
a = df.groupby('Team')['Bonus'].mean()
df['Bonus'] = df['Bonus'].fillna(df['Team'].map(a))



Fill missing Sport_Type with "Other".


In [10]:
c = 'Other'
df['Sport_Type'] = df['Sport_Type'].fillna(c)


Fill missing Venue values with "Unspecified".

In [11]:
h = 'Unspecified'
df['Venue'] = df['Venue'].fillna(h)


Remove rows where Match_Fee is less than or equal to 0.



In [12]:
df = df[df['Match_Fee'] > 0].reset_index(drop=True)


Remove rows where Bonus is negative.

In [13]:
n = df['Bonus'] >= 0
df = df[n]


Remove rows where Age < 10 or Age > 60.

In [14]:
t = (df['Age'] >= 10) & (df['Age'] <= 60)
df = df[t]


Remove rows where Feedback_Score is outside the range 1–5.



In [15]:
u = (df['Feedback_Score'] >= 1) & (df['Feedback_Score'] <= 5)
df = df[u]


Remove rows where Ticket_Sales > 50,000 (possible entry error).

In [16]:
a = df['Ticket_Sales'] <= 50000
df = df[a].reset_index(drop=True)


Fix invalid Match_Date formats (mixed date formats).




In [17]:
d = pd.to_datetime(df['Match_Date'], errors='coerce')
df['Match_Date'] = d.dt.strftime('%Y-%m-%d')


Correct typos in Gender column (Mle, femal, mal, etc.).

In [18]:
h = {'male': 'Male', 'MALE': 'Male', 'Xale': 'Male', 'Yale': 'Male',
     'Zale': 'Male', 'XALE': 'Male',
    'female': 'Female', 'FEMALE': 'Female', 'Xemale': 'Female',
    'Yemale': 'Female', 'Zemale': 'Female', 'XEMALE': 'Female',
    'ZEMALE': 'Female',
    'other': 'Other', 'OTHER': 'Other', 'Yther': 'Other',
    'Zther': 'Other', 'Xther': 'Other', 'YTHER': 'Other'}

df['Gender'] = df['Gender'].replace(h)


Correct typos in Payment_Mode column (creadit card, upiid, etc.).



In [19]:
n = {'Credit Card': 'Credit Card', 'credit card': 'Credit Card',
    'CREDIT CARD': 'Credit Card', 'Xredit Xard': 'Credit Card',
    'Yredit Yard': 'Credit Card', 'Zredit Zard': 'Credit Card',
    'XREDIT XARD': 'Credit Card', 'YREDIT YARD': 'Credit Card',
    'ZREDIT ZARD': 'Credit Card',

    'Cash': 'Cash', 'cash': 'Cash', 'CASH': 'Cash',
    'Xash': 'Cash', 'Yash': 'Cash', 'Zash': 'Cash',
    'YASH': 'Cash', 'ZASH': 'Cash',

    'UPI': 'UPI', 'upi': 'UPI',
    'XPI': 'UPI', 'YPI': 'UPI', 'ZPI': 'UPI',
    'Xpi': 'UPI', 'Ypi': 'UPI',

    'Online': 'Online', 'online': 'Online',
    'ONLINE': 'Online', 'Xnline': 'Online',
    'Ynline': 'Online', 'Znline': 'Online',
    'YNLINE': 'Online'}
df['Payment_Mode'] = df['Payment_Mode'].replace(n)


Standardize Win_Status values to ["Win", "Lose", "Draw"].

In [20]:
h = {'Win': 'Win', 'WIN': 'Win', 'win': 'Win',
    'Zin': 'Win', 'Yin': 'Win', 'Xin': 'Win',
    'ZIN': 'Win', 'YIN': 'Win',

    'Lose': 'Lose', 'lose': 'Lose', 'LOSE': 'Lose',
    'Xose': 'Lose', 'Yose': 'Lose', 'Zose': 'Lose',
    'XOSE': 'Lose', 'YOSE': 'Lose', 'ZOSE': 'Lose',

    'Draw': 'Draw', 'draw': 'Draw', 'DRAW': 'Draw',
    'Xraw': 'Draw', 'Yraw': 'Draw', 'Zraw': 'Draw',
    'XRAW': 'Draw', 'YRAW': 'Draw'}
df['Win_Status'] = df['Win_Status'].replace(h)


Replace unrealistic Sponsorship_Amount values (< 0 or > 1,000,000) with median

In [21]:
t = df['Sponsorship_Amount'].median()
df['Sponsorship_Amount'] = df['Sponsorship_Amount'].where((df['Sponsorship_Amount'] >= 0) & (df['Sponsorship_Amount'] <= 1_000_000),t)


Fetch all matches where Match_Fee > 4000.



In [22]:
t = df['Match_Fee'] > 4000
df[t]


,Player_ID,Player_Name,Gender,Age,Country,Team,Position,Sport_Type,Match_ID,Tournament,...,Bonus,Sponsorship_Amount,Ticket_Sales,Fan_ID,Ticket_Type,Payment_Mode,Feedback_Score,Coach_Name,Win_Status,Weather
4,P4657,Liu,Other,25,Germany,Team C,Forward,Basketball,M55549,World Cup,...,28.75,9181.16,2383.16,F24847,VVIP,Credit Card,3.0,Brown,Win,Windy
9,P7912,Mike,Other,40,South Africa,Team B,Bowler,Cricket,M60157,Friendly,...,220.50,8815.61,9130.40,F29794,VIP,Online,2.0,Lee,Lose,Sunny
12,P2535,John,Female,25,South Africa,Team A,Goalkeeper,basketball,M65085,League,...,677.79,8726.81,6165.10,F66389,VIP,Cash,1.0,Patel,Draw,Windy
15,P9279,Liu,Other,35,South Africa,Team D,Defender,Tennis,M77300,Friendly,...,347.86,1833.51,8499.91,F13053,Regular,Credit Card,4.0,Lee,Draw,Windy
20,P4611,David,Other,36,Australia,Team B,Bowler,Cricket,M49747,Championship,...,1283.58,4019.68,225.84,F70378,VVIP,Cash,2.0,Patel,Lose,Rainy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4016,P2517,Liu,Male,37,USA,Team D,Forward,Cricket,M53972,Championship,...,1724.96,4613.46,1444.16,F40699,VVIP,Online,4.0,Lee,Win,NaN
4025,P1598,Maria,Other,41,UK,Team A,Defender,Cricket,M21339,Friendly,...,1535.40,5422.66,8980.71,F91443,Regular,Online,2.0,Brown,Win,Rainy
4028,P9558,John,Male,20,Australia,Team A,Defender,Football,M88705,World Cup,...,567.71,8666.15,2189.81,F98701,VVIP,Credit Card,4.0,Brown,Win,Rainy
4031,P1755,John,Other,38,Australia,Team C,Defender,Basketball,M71304,League,...,1798.02,3416.81,9347.12,F27842,VIP,Credit Card,5.0,Patel,NaN,Cloudy


Fetch all players with Feedback_Score < 3.



In [23]:
f=df[df['Feedback_Score']<3]
print(f)


     Player_ID Player_Name  Gender  Age       Country    Team    Position  \
1        P1409       Maria  Female   34       Germany  Team D  Goalkeeper   
6        P2679      Carlos    Male   43        Brazil  Team E     Batsman   
7        P9935        John    Male   35       Germany  Team B     Batsman   
8        P2424       Maria   Other   21           USA  Team E  Goalkeeper   
9        P7912        Mike   Other   40  South Africa  Team B      Bowler   
...        ...         ...     ...  ...           ...     ...         ...   
4022     P6572        John    Male   25           USA  Team A  Goalkeeper   
4025     P1598       Maria   Other   41            UK  Team A    Defender   
4026     P8526        Amit    Male   35  South Africa  Team D     Batsman   
4029     P2426      Carlos  Female   18            UK  Team E    Defender   
4030     P8649         Liu  Female   23  South Africa  Team E     Forward   

      Sport_Type Match_ID    Tournament  ...    Bonus Sponsorship_Amount  \

Fetch all players from India who played more than 10 matches.



In [24]:
p=df[(df['Score']>100) & (df['Country']=='india')]
print(p)

     Player_ID Player_Name  Gender  Age Country    Team  Position  Sport_Type  \
968      P7464        Mike  Female   19   india  Team A    Bowler      Tennis   
1557     P3102       David  Female   36   india  Team C  Defender  Basketball   
2427     P5693        Sara  Female   30   india  Team E  Defender  Basketball   

     Match_ID    Tournament  ...    Bonus Sponsorship_Amount Ticket_Sales  \
968    M74239  Championship  ...  1994.86            6969.03      3405.59   
1557   M71460        League  ...   555.92            6111.83      9947.84   
2427   M39193     World Cup  ...  1791.03            7714.44      7750.96   

      Fan_ID  Ticket_Type  Payment_Mode  Feedback_Score  Coach_Name  \
968   F74992          VIP        Online             2.0         Lee   
1557  F67982         VVIP          Cash             5.0    Anderson   
2427  F62626          VIP        Online             5.0       Patel   

      Win_Status  Weather  
968         Draw    Rainy  
1557         Win   Cloudy

Fetch all VIP ticket buyers where Ticket_Sales > 5000.



In [25]:
t=df[(df['Ticket_Type']=='VIP')&(df['Ticket_Sales']>5000)]
print(t)

     Player_ID Player_Name  Gender  Age       Country    Team    Position  \
8        P2424       Maria   Other   21           USA  Team E  Goalkeeper   
9        P7912        Mike   Other   40  South Africa  Team B      Bowler   
12       P2535        John  Female   25  South Africa  Team A  Goalkeeper   
18       P9928         Liu    Male   38           USA  Team E  Goalkeeper   
21       P8359        Amit   Other   35     Australia  Team E    Defender   
...        ...         ...     ...  ...           ...     ...         ...   
4012     P5958      Carlos  Female   38           usa  Team E    Defender   
4018     P9290        Amit  Female   24           USA  Team E     Batsman   
4027     P7350         Liu  Female   19            UK  Team B    Defender   
4030     P8649         Liu  Female   23  South Africa  Team E     Forward   
4031     P1755        John   Other   38     Australia  Team C    Defender   

      Sport_Type Match_ID    Tournament  ...    Bonus Sponsorship_Amount  \


Fetch all matches where Win_Status = "Win" and Score > 100.



In [26]:
i=df[(df['Win_Status']=='Win')&(df['Score']>100)]
print(i)

     Player_ID Player_Name  Gender  Age       Country    Team    Position  \
4        P4657         Liu   Other   25       Germany  Team C     Forward   
14       P4811        Amit  Female   38           USA  Team B      Bowler   
16       P1434      Carlos  Female   18           USA  Team B     Forward   
68       P8573       Priya  Female   23  South Africa  Team D    Defender   
71       P4598        Amit    Male   16       Germany  Team E  Goalkeeper   
...        ...         ...     ...  ...           ...     ...         ...   
3977     P9069        Sara    Male   29        Brazil  Team C     Batsman   
3991     P1477       David  Female   36  South Africa  Team C      Bowler   
4014     P9603        John   Other   35         India  Team D     Forward   
4015     P4311       Maria    Male   41     Australia  Team E     Forward   
4025     P1598       Maria   Other   41            UK  Team A    Defender   

      Sport_Type Match_ID    Tournament  ...    Bonus Sponsorship_Amount  \

Fetch all cricket players with Wickets > 3.




In [27]:
t=df[(df['Player_Name'].notnull())&(df['Wickets']>3)]
print(t)

     Player_ID Player_Name  Gender  Age       Country    Team    Position  \
0        P2824        Mike    Male   21            UK  Team E     Forward   
2        P5506       David    Male   43  South Africa  Team D  Goalkeeper   
10       P1520       Maria    Male   33     Australia  Team B      Bowler   
14       P4811        Amit  Female   38           USA  Team B      Bowler   
32       P2519       Maria   Other   40           USA  Team E     Batsman   
...        ...         ...     ...  ...           ...     ...         ...   
4024     P6084         Liu  Female   39        Brazil  Team E      Bowler   
4028     P9558        John    Male   20     Australia  Team A    Defender   
4029     P2426      Carlos  Female   18            UK  Team E    Defender   
4031     P1755        John   Other   38     Australia  Team C    Defender   
4033     P1368       Maria     NaN   21            UK  Team B    Defender   

      Sport_Type Match_ID    Tournament  ...    Bonus Sponsorship_Amount  \

Fetch all football players with Goals >= 2.

In [28]:
y=df[(df['Player_Name'].notnull())&(df['Goals']>=2)]
y[['Goals','Player_Name']]

,Goals,Player_Name
0,4.0,Mike
1,4.0,Maria
2,4.0,David
3,4.0,Sara
6,3.0,Carlos
...,...,...
4025,4.0,Maria
4026,3.0,Amit
4027,4.0,Liu
4031,3.0,John


Fetch all matches with total Bonus + Match_Fee > 6000.

In [29]:
t=df[df['Bonus']+df['Match_Fee']>6000]
print(t)

     Player_ID Player_Name  Gender  Age    Country    Team    Position  \
38       P1711         Liu   Other   41    Germany  Team D      Bowler   
44       P5803         Liu    Male   29     Brazil  Team B      Bowler   
64       P3803        Sara    Male   42    GERMANY  Team E      Bowler   
69       P7216         Liu   Other   40      India  Team E     Batsman   
86       P3340      Carlos     NaN   22    Germany  Team A     Forward   
...        ...         ...     ...  ...        ...     ...         ...   
3933     P7292       David    Male   43     Brazil  Team B     Forward   
3982     P6468       David  Female   39      India  Team E  Goalkeeper   
4016     P2517         Liu    Male   37        USA  Team D     Forward   
4025     P1598       Maria   Other   41         UK  Team A    Defender   
4031     P1755        John   Other   38  Australia  Team C    Defender   

      Sport_Type Match_ID    Tournament  ...    Bonus Sponsorship_Amount  \
38       Cricket   M55598      Frie

Fetch all players whose average Rating < 4.



In [30]:
t=df.loc[df['Rating']<4,'Rating'].mean()
print(t)

2.500273578113751


Fetch all matches in Rainy weather conditions

In [31]:
t=df[(df['Match_ID'].notnull()) & (df['Weather']=='Rainy')]
print(t[['Match_ID','Weather']])

     Match_ID Weather
7      M10217   Rainy
14     M27004   Rainy
16     M64419   Rainy
17     M82458   Rainy
20     M49747   Rainy
...       ...     ...
4022   M62935   Rainy
4025   M21339   Rainy
4026   M34565   Rainy
4027   M60673   Rainy
4028   M88705   Rainy

[943 rows x 2 columns]


Concatenate Player id and Gender

In [32]:
t=df['Player_ID']+"-" + df['Gender']
print(t)

0         P2824-Male
1       P1409-Female
2         P5506-Male
3         P5012-Male
4        P4657-Other
            ...     
4030    P8649-Female
4031     P1755-Other
4032      P2689-Male
4033             NaN
4034      P6114-Male
Length: 4035, dtype: object


Concatenate MatchID and FeedbackScore

In [33]:
f=df['Match_ID'].astype(str)+"-" + df['Feedback_Score'].astype(str)
print(f)

0       M65327-5.0
1       M93279-1.0
2       M93088-4.0
3       M84016-5.0
4       M55549-3.0
           ...    
4030    M33827-2.0
4031    M71304-5.0
4032    M69868-3.0
4033    M62739-5.0
4034    M38762-5.0
Length: 4035, dtype: object


Compain playerID and fanID as unified record

In [34]:
df['unfied_Record']=df['Player_ID']+"-" + df['Fan_ID']
print(df['unfied_Record'])

0       P2824-F83977
1       P1409-F51266
2       P5506-F74976
3       P5012-F36267
4       P4657-F24847
            ...     
4030    P8649-F31235
4031    P1755-F27842
4032    P2689-F23798
4033    P1368-F69272
4034    P6114-F24891
Name: unfied_Record, Length: 4035, dtype: object


Calculate total earning per player (Match_Fee + Bonus + Sponsorship_Amount).



In [35]:
df['g']=df['Match_Fee']+df['Bonus']+df['Sponsorship_Amount']
y=(df[['Player_ID','g']])
print(y)

     Player_ID         g
0        P2824   9674.93
1        P1409   8751.73
2        P5506   6559.52
3        P5012  11808.05
4        P4657  13369.51
...        ...       ...
4030     P8649  10337.05
4031     P1755   9962.80
4032     P2689   9394.68
4033     P1368  11604.49
4034     P6114  12034.88

[4035 rows x 2 columns]


Calculate average rating per sport type.




In [36]:
y=df.groupby('Sport_Type')['Rating'].mean()
print(y)

Sport_Type
BASKETBALL    4.965652
Basketball    5.342434
CRICKET       5.289048
Cricket       5.615248
FOOTBALL      5.494615
Football      5.522750
TENNIS        6.203810
Tennis        5.422371
Xasketball    4.528000
Xennis        4.551429
Xootball      7.346250
Xricket       5.510714
YENNIS        6.320000
Yasketball    4.545000
Yennis        5.821667
Yootball      4.779231
Yricket       7.104000
ZOOTBALL      2.560000
Zasketball    4.555000
Zennis        5.638000
Zootball      4.476923
Zricket       3.848000
basketball    5.389048
cricket       5.290667
football      6.142500
tennis        5.708462
Name: Rating, dtype: float64


Identify players who only played one type of sport.

In [37]:
t=df.groupby('Rating')['Sport_Type'].nunique().eq(1)
print(t,'Rating','Sport_type')

Rating
1.00     False
1.01     False
1.02     False
1.03     False
1.04     False
         ...  
9.96     False
9.97     False
9.98     False
9.99     False
10.00    False
Name: Sport_Type, Length: 890, dtype: bool Rating Sport_type


Identify top 10 players by total sponsorship earnings.




In [38]:
top10=df.groupby('Player_Name')['Sponsorship_Amount'].sum().nlargest(10).reset_index()
print(top10)

  Player_Name  Sponsorship_Amount
0       David          2636567.11
1         Liu          2547624.09
2       Maria          2533598.54
3        Sara          2531212.24
4       Priya          2455783.99
5        Amit          2435579.09
6        Mike          2406579.89
7        John          2393191.72
8      Carlos          2271123.00


Fench all playername who have no coach

In [39]:
t=df['Coach_Name'].isnull()
y=df.loc[t,'Player_Name']
y

,Player_Name


Identify venues used by more than one sport type.




In [40]:
t=df.groupby('Venue')['Sport_Type'].nunique()
print(t)

Venue
Arena 2        24
Court 4        20
Ground 3       22
Stadium 1      23
Unspecified    17
Name: Sport_Type, dtype: int64


Add a column Total_Earning = Match_Fee + Bonus + Sponsorship_Amount.




In [41]:
df['Total_Earning']=(df['Match_Fee']+ df['Bonus'] +df['Sponsorship_Amount'])
df[['Total_Earning','Match_ID']]

,Total_Earning,Match_ID
0,9674.93,M65327
1,8751.73,M93279
2,6559.52,M93088
3,11808.05,M84016
4,13369.51,M55549
...,...,...
4030,10337.05,M33827
4031,9962.80,M71304
4032,9394.68,M69868
4033,11604.49,M62739


Add a column Performance_Score = Score + (Assists * 2) + (Goals * 3) + (Wickets * 4).



In [42]:
df['Performence_Score']=(df['Score'] +(df['Assists']*2) +(df['Goals']*3)+(df['Wickets']*4))
df['Performence_Score']

,Performence_Score
0,45.0
1,84.0
2,128.0
3,46.0
4,137.0
...,...
4030,56.0
4031,130.0
4032,50.0
4033,43.0


Add a column Revenue = Ticket_Sales + Merch_Sales (create Merch_Sales if missing).



In [43]:
if 'Merch_Sales' not in df.columns:
    df['Merch_Sales']=0


In [44]:
df['Revenue']=(df['Ticket_Sales']+df['Merch_Sales'])
df['Revenue']

,Revenue
0,3413.96
1,9388.07
2,4737.38
3,9372.68
4,2383.16
...,...
4030,8718.13
4031,9347.12
4032,1926.08
4033,9102.33


Create an Age_Group column:
<20: Junior
20–30: Adult
31–45: Senior

In [45]:
def Age_group(Age):
  if Age <20:
    return 'Junior'
  elif 20<=Age<=30:
    return 'Adult'
  elif 31<=Age<=45:
    return 'Senior'
df['Age_group']=df['Age'].apply(Age_group)

Create a Performance_Level column:
Rating < 4 → “Low”
4–7 → “Medium”
7 → “High”

In [46]:
def Performance_level(Rating):
  if Rating<4:
    return 'low'
  elif 4<=Rating<=7:
    return 'medium'
  else:
    return 'high'
df['Performance_level']=df['Rating'].apply(Performance_level)


Encode Gender numerically (Male = 1, Female = 2, Other = 3).




In [47]:
df['gender_numeric']=df['Gender'].map({'Male':1,'Female':2,'Others':3})

Encode Win_Status numerically (Win = 1, Draw = 0.5, Lose = 0).



In [48]:
df['Win_status_numeric']=df['Win_Status'].map({'Win':1,'Lose':2,'Draw':3})

Standardize Country names (capitalize first letters).



In [49]:
df['Country']=df['Country'].str.title()

Remove extra spaces from all text columns.

In [50]:
col=['Country','Player_Name','Gender','Team','Position','Sport_Type','Tournament','Opponent_Team','Venue','City','Assists','Ticket_Type','Payment_Mode','Coach_Name','Win_Status','Weather']
df[col]=df[col].astype(str)
for c in col:
  df[c]=df[c].str.strip()

Convert all string data to lowercase for text comparison.


In [51]:
for c in df.columns:
  if df[c].dtype=="object":
    df[c]=df[c].astype(str).str.lower()

Find top 5 players by total earnings.



In [52]:
top5 = (df.groupby("Player_ID")["Match_Fee"].sum().sort_values(ascending=False).head(5))
print(top5)

Player_ID
p4245    4999.73
p5092    4998.96
p1701    4998.02
p8216    4998.02
p8591    4995.84
Name: Match_Fee, dtype: float64



Find top 5 teams by total runs or goals scored.



In [53]:
top5_teams = (df.groupby("Team")["Score"].sum().sort_values(ascending=False).head(5))
print(top5_teams)


Team
team a    63946.0
team e    63922.0
team b    57856.0
team d    57417.0
team c    55770.0
Name: Score, dtype: float64


Find average feedback score per sport type.



In [54]:
y = (df.groupby("Sport_Type")["Feedback_Score"].mean().reset_index().sort_values("Feedback_Score", ascending=False))
print(y)


    Sport_Type  Feedback_Score
15     zricket        3.600000
11     yricket        3.400000
10    yootball        3.307692
6     xootball        3.125000
7      xricket        3.071429
3       tennis        3.030644
0   basketball        3.026289
8   yasketball        3.000000
14    zootball        3.000000
2     football        2.981799
1      cricket        2.959764
4   xasketball        2.800000
9       yennis        2.750000
5       xennis        2.571429
12  zasketball        2.500000
13      zennis        1.800000


Find average bonus per team.



In [55]:
avg_bonus = (df.groupby("Team")["Bonus"].mean().reset_index().sort_values("Bonus", ascending=False))
print(avg_bonus)


     Team        Bonus
2  team c  1021.709869
1  team b   996.965508
3  team d   994.465139
4  team e   989.853254
0  team a   983.874132


Find the most common payment mode among fans.



In [56]:
most_common_payment = df["Payment_Mode"].value_counts().idxmax()
print(most_common_payment)

upi


Identify the player with the highest single match score.




In [57]:
top_player= df.loc[df["Score"].idxmax()]
print(top_player["Player_ID"], top_player["Score"])


p9830 149.0


Find players who have played more than 15 matches.



In [58]:
players_15plus = (df.groupby("Player_ID").size().reset_index(name="Match_count").query("Match_count > 15"))
print(players_15plus)


Empty DataFrame
Columns: [Player_ID, Match_count]
Index: []


Sort players by their total minutes played (descending).



In [59]:
players_sorted = (df.groupby("Player_ID")["Minutes_Played"].sum().reset_index().sort_values("Minutes_Played", ascending=False))
print(players_sorted)


     Player_ID  Minutes_Played
2987     p7688           119.0
3896     p9698           119.0
1681     p4825           119.0
1732     p4941           119.0
2730     p7116           119.0
...        ...             ...
1504     p4404             0.0
1553     p4530             0.0
1567     p4562             0.0
1552     p4527             0.0
3285     p8357             0.0

[4035 rows x 2 columns]


Identify the most frequently played venue.



In [60]:
t = df['Venue'].value_counts().head(1)
print(t)


Venue
stadium 1    984
Name: count, dtype: int64


Group data by Country and Sport_Type, and calculate total ticket sales.




In [61]:
grouped = df.groupby(['Country', 'Sport_Type'])['Ticket_Sales'].sum().reset_index()
print(grouped)


       Country  Sport_Type  Ticket_Sales
0    australia  basketball     642682.79
1    australia     cricket     630827.79
2    australia    football     591186.39
3    australia      tennis     746667.85
4    australia  xasketball       3960.60
..         ...         ...           ...
152        zsa      tennis       1773.43
153  zustralia  basketball      17699.68
154  zustralia     cricket      13724.13
155  zustralia    football       9917.30
156  zustralia      tennis      26850.39

[157 rows x 3 columns]


Detect inconsistent date formats in Match_Date and fix them to YYYY-MM-DD.



In [62]:
df['Match_Date'] = pd.to_datetime(df['Match_Date'], errors='coerce').dt.strftime('%Y-%m-%d')
df['Match_Date']

,Match_Date
0,2023-05-05
1,NaN
2,NaN
3,NaN
4,NaN
...,...
4030,NaN
4031,NaN
4032,NaN
4033,2021-07-08


Identify players who appear with different Countries (data inconsistency).



In [63]:
inconsistent_players = df.groupby('Player_ID')['Country'].nunique().gt(1)

player_names = inconsistent_players[inconsistent_players].index
print(player_names)



Index([], dtype='object', name='Player_ID')


Detect outliers in Sponsorship_Amount using IQR method.



In [64]:
Q1 = df['Sponsorship_Amount'].quantile(0.25)
Q3 = df['Sponsorship_Amount'].quantile(0.75)
IQR = Q3 - Q1

outlier_values = df.loc[
    (df['Sponsorship_Amount'] < Q1 - 1.5 * IQR) |
    (df['Sponsorship_Amount'] > Q3 + 1.5 * IQR),
    'Sponsorship_Amount']

print(outlier_values)


Series([], Name: Sponsorship_Amount, dtype: float64)


Replace blank Player_Name values with "Anonymous".



In [65]:
df['Player_Name'] = df['Player_Name'].replace('', 'Anonymous')
df['Player_Name']


,Player_Name
0,mike
1,maria
2,david
3,sara
4,liu
...,...
4030,liu
4031,john
4032,carlos
4033,maria



Validate Payment_Mode entries (must be from the list ["Cash", "Credit Card", "UPI", "Online"]).



In [66]:
valid_modes = ["Cash", "Credit Card", "UPI", "Online"]
invalid_payments = df.loc[~df['Payment_Mode'].isin(valid_modes),['Player_Name', 'Payment_Mode']]
print(invalid_payments)


     Player_Name Payment_Mode
0           mike  credit card
1          maria         cash
2          david          upi
3           sara          upi
4            liu  credit card
...          ...          ...
4030         liu         cash
4031        john  credit card
4032      carlos  credit card
4033       maria         cash
4034        amit         cash

[4035 rows x 2 columns]


Identify and fix misspelled Sport_Type (e.g., “footbal”, “basktball”).



In [67]:
valid_sports = ["Cricket", "Football", "Basketball", "Tennis"]
misspelled = df[~df['Sport_Type'].str.title().isin(valid_sports)][['Player_Name', 'Sport_Type']]
print(misspelled)


     Player_Name  Sport_Type
46          mike  zasketball
56          sara      yennis
102        david    xootball
131         john     zricket
149         amit     xricket
...          ...         ...
3881        john  zasketball
3955        sara    zootball
4014        john  xasketball
4030         liu     xricket
4034        amit     yricket

[114 rows x 2 columns]


Verify all Fan_ID values are unique.



In [68]:
df = df.drop_duplicates(subset='Fan_ID')
df['Fan_ID'].is_unique


True

Remove leading/trailing spaces from Player_Name, Coach_Name, and Country.



In [69]:
df['Player_Name'] = df['Player_Name'].str.strip()
df['Coach_Name'] = df['Coach_Name'].str.strip()
df['Country'] = df['Country'].str.strip()


Convert all Match_Date to datetime and sort chronologically.



In [70]:
df['Match_Date'] = pd.to_datetime(df['Match_Date'], errors='coerce')
df = df.sort_values(by='Match_Date')


Drop all irrelevant columns after cleaning (like Remarks, Seat_Number, etc.).

In [71]:
df=df.drop(columns=['Weather'])



Remove all column with null value from the data set

In [72]:
df=df.dropna(axis=0)

In [73]:
from google.colab import files
df.to_excel("cleaned_file.xlsx", index=False)
files.download("cleaned_file.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>